In [1]:
with open('../data/Korean_movie_reviews_2016.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[0], int(doc[1])) for doc in docs if len(doc) == 2]
    texts, labels = zip(*docs)
    # 둘을 분리해서 별도의 list 변수로 저장

In [2]:
docs_words = [doc.strip().split() for doc in texts]

In [3]:
print(docs_words[:2])

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_docs = [TaggedDocument(doc, tags=[i]) for i, doc in enumerate(docs_words) if doc != None]

In [5]:
tagged_docs[:2]

In [6]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec(tagged_docs, vector_size=100, min_count=3, epochs=100, dm=1, negative=5, 
               alpha=0.01)

In [7]:
model = Doc2Vec(tagged_docs, vector_size=100, min_count=3, window=3, epochs=1000, dm=1, negative=5, 
               alpha=0.001)

In [8]:
len(model.dv)

In [9]:
len(texts)

In [10]:
len(labels)

In [11]:
texts[0]

In [12]:
import numpy as np
np.dot(model.dv[89881], model.dv[0])/(np.linalg.norm(model.dv[89881])*np.linalg.norm(model.dv[0]))

In [13]:
for id, sim in model.dv.most_similar(0, topn=10):
    print('reivew ID: {}, review: {}, similarity: {}'.format(id, texts[id],sim))

In [14]:
docs_vectors = []
for i in range(len(texts)):
    docs_vectors.append(model.dv[i])

In [15]:
len(docs_vectors)

In [16]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(docs_vectors, labels, test_size=0.2, random_state=0)

In [17]:
print(len(train_features), len(train_labels))

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
lr2 = LogisticRegression(C=1, penalty='l2', solver='saga') 
lr2.fit(train_features, train_labels) 
pred_labels = lr2.predict(test_features)

In [20]:
from sklearn.metrics import accuracy_score
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

In [21]:
lr1 = LogisticRegression(C=1, penalty='l1', solver='saga') 
lr1.fit(train_features, train_labels) 
pred_labels = lr1.predict(test_features)
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))